In [1]:
#Librerías

import os
import pandas as pd
import re
import glob
from collections import defaultdict
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import norm, cauchy, levy_stable
from concurrent.futures import ThreadPoolExecutor
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
import csv
import pyarrow.parquet as pq

In [2]:
import pyarrow.parquet as pq
import pandas as pd
import torch
from torch.utils.data import Dataset

class ResiduosDataset(Dataset):
    def __init__(self, index_path, split):
        self.index = pd.read_csv(index_path)
        if isinstance(split, list):
            self.index = self.index[self.index['split'].isin(split)]
        else:
            self.index = self.index[self.index['split'] == split]
        self.index = self.index.reset_index(drop=True)

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        meta = self.index.iloc[idx]
        table = pq.read_table(meta.path_archivo,
                              columns=["Return", "Alpha", "Beta", "Gamma", "Delta"],
                              filters=[("Rep", "=", meta.rep)])
        serie = table.column("Return").to_numpy().astype("float64")
        alpha = meta.alpha
        beta = meta.beta
        nombre = f"{alpha}_{beta}_rep{meta.rep}"
        return serie, torch.tensor([alpha, beta], dtype=torch.float64), nombre

In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
INDEX_PATH= r"D:\OneDrive\OneDrive - Universidad Técnica Federico Santa María\2025\Paper CAPM\Simulaciones4\index_master.csv"

#split dataset
ds_pretrain = ResiduosDataset(index_path=INDEX_PATH, split="pre_train")
ds_pretrain = ResiduosDataset(index_path=INDEX_PATH, split="pre_train")
ds_train_raw = ResiduosDataset(index_path=INDEX_PATH, split="train")
ds_train = ConcatDataset([ds_pretrain, ds_train_raw])
ds_val      = ResiduosDataset(index_path=INDEX_PATH, split="val")
ds_test     = ResiduosDataset(index_path=INDEX_PATH, split="test")

#dloader
loader_pretrain = DataLoader(ds_pretrain, batch_size=128, shuffle=True, num_workers=0)
loader_train    = DataLoader(ds_train, batch_size=128, shuffle=True, num_workers=0)
loader_val      = DataLoader(ds_val, batch_size=128, shuffle=False, num_workers=0)
loader_test     = DataLoader(ds_test, batch_size=128, shuffle=False, num_workers=0)

In [4]:
def stable_pdf_series_infinity(x, alpha, beta, max_coef, device=None):
    """
    Serie infinita para estimar la PDF de una distribución estable con α > 0.5.
    """
    eps = 1e-16
    if device is None:
        device = x.device if torch.is_tensor(x) else "cpu"

    # Asegurar float64
    x = torch.as_tensor(x, dtype=torch.float64, device=device)
    alpha = torch.as_tensor(alpha, dtype=torch.float64, device=device)
    beta = torch.as_tensor(beta, dtype=torch.float64, device=device)

    pi = torch.tensor(torch.pi, dtype=torch.float64, device=device)
    zeta = -beta * torch.tan(pi * alpha / 2)
    x_minus_zeta = x - zeta

    if torch.any(x_minus_zeta <= 0):
        raise ValueError("x - zeta contiene valores no positivos, lo que genera potencias negativas indefinidas.")

    k = torch.arange(0, max_coef + 1, dtype=torch.float64, device=device)  # 0 a max_coef
    gamma_part = torch.exp(torch.lgamma(alpha * (k + 1)) - torch.lgamma(k + 1))  # gamma / k!
    sin_part = torch.sin((pi / 2 * alpha - torch.atan(zeta)) * (k + 1))
    sqrt_1pz2 = torch.sqrt(1 + zeta ** 2)
    geometric_part = sqrt_1pz2 ** (k + 1)

    # x-dependent part (potencias negativas acumuladas)
    x_power = (x_minus_zeta.unsqueeze(-1)) ** (-alpha * (k + 1))  # shape: [len(x), len(k)]
    x_div = 1.0 / (x_minus_zeta.unsqueeze(-1))  # x - zeta en el denominador

    # Signo alternado (-1)^k
    signs = (-1.0) ** k

    # Producto escalar de todos los términos
    terms = signs * gamma_part * geometric_part * sin_part  # shape: [len(k)]
    full_terms = terms * x_power * x_div  # broadcasting sobre x

    val = full_terms.sum(dim=-1) * alpha / pi  # sum sobre coeficientes

    # Validación
    if torch.isnan(val).any() or (~torch.isfinite(val)).any():
        raise ValueError(f"[stable_pdf_series_infinity] Valores inválidos — α={alpha.item()}, β={beta.item()}")

    return val



In [5]:
def stable_pdf_fourier_integral(x, alpha, beta):
    """
    Estima la densidad de una distribución estable usando el método de integral de Fourier
    según Ament & O’Neill.
    """
    eps = 1e-16
    device = x.device if torch.is_tensor(x) else "cpu"
    dtype = torch.float64

    x = torch.as_tensor(x, dtype=dtype, device=device)
    alpha = torch.as_tensor(alpha, dtype=dtype, device=device)
    beta = torch.as_tensor(beta, dtype=dtype, device=device)

    if alpha >= 1.1:
        gx = torch.tensor([
            7.0370416932351169e-06, 8.9457728587460458e-05, 4.5195717164507891e-04, 1.4387800013186535e-03,
            3.4238229401029920e-03, 6.6679664869140516e-03, 1.1244527018077863e-02, 1.7062429312798121e-02,
            2.3941528921398801e-02, 3.1681730492178345e-02, 4.0101739779817722e-02, 4.9052345082266816e-02,
            5.8416313367213923e-02, 6.8103426617886029e-02, 7.8044751687152913e-02, 8.8187621674808730e-02,
            9.8491651958493867e-02, 1.0892570482430752e-01, 1.1946561554032360e-01, 1.3009249854990793e-01,
            1.4079148624023027e-01, 1.5155078763117422e-01, 1.6236098315732694e-01, 1.7321449381516493e-01,
            1.8410517939724058e-01, 1.9502803262698309e-01, 2.0597894484676815e-01, 2.1695452535829091e-01,
            2.2795196121330644e-01, 2.3896890767984083e-01, 2.5000340211253524e-01, 2.6105379579367261e-01,
            2.7211869966031971e-01, 2.8319694083099400e-01, 2.9428752758178356e-01, 3.0538962098220246e-01,
            3.1650251181003752e-01, 3.2762560167565286e-01, 3.3875838752038662e-01, 3.4990044884671112e-01,
            3.6105143718373622e-01, 3.7221106735260195e-01, 3.8337911022287513e-01, 3.9455538670580470e-01,
            4.0573976283971641e-01, 4.1693214574735116e-01, 4.2813248020546890e-01, 4.3934074598515077e-01,
            4.5055695570150323e-01, 4.6178115355311344e-01, 4.7301341376196310e-01, 4.8425383952216777e-01,
            4.9550256288736721e-01, 5.0675974549192959e-01, 5.1802557928302240e-01, 5.2930028492144732e-01,
            5.4058411161129039e-01, 5.5187734378208209e-01, 5.6318030393397900e-01, 5.7449335812231850e-01,
            5.8581688888130956e-01, 5.9715132998513110e-01, 6.0849716286556388e-01, 6.1985498063016387e-01,
            6.3122540832012675e-01, 6.4260904482606040e-01, 6.5400647463066774e-01, 6.6541875634974845e-01,
            6.7684685622122265e-01, 6.8829256028106156e-01, 6.9975558376055846e-01, 7.1123699066990020e-01,
            7.2273896702242169e-01, 7.3426487948155317e-01, 7.4582269874591844e-01, 7.5740470496696566e-01,
            7.6901033051191892e-01, 7.8063124635092274e-01, 7.9230651366491101e-01, 8.0410492102734399e-01,
            8.1612625247941939e-01, 8.2818135034865736e-01, 8.4120303825748510e-01, 8.5677958040655922e-01,
            8.7803721055848216e-01, 9.1078416473987345e-01
        ], dtype=dtype, device=device)

        gw = torch.tensor([
            2.4123585761838668e-05, 1.7469393088180406e-04, 6.0975563988952377e-04, 1.4288650277602072e-03,
            2.5860145358909906e-03, 3.9139638862970855e-03, 5.2225620441390059e-03, 6.3820155329709829e-03,
            7.3422109186047540e-03, 8.1077876105023049e-03, 8.7073841240531413e-03, 9.1743792256825593e-03,
            9.5386194524040412e-03, 9.8242023344950434e-03, 1.0049736076644946e-02, 1.0229323237713391e-02,
            1.0373585169887569e-02, 1.0490520985968452e-02, 1.0586173608364573e-02, 1.0665129967020341e-02,
            1.0730891859179584e-02, 1.0786149736593508e-02, 1.0832984766811224e-02, 1.0873018221674781e-02,
            1.0907522259639847e-02, 1.0937502403629278e-02, 1.0963759233467768e-02, 1.0986934773375319e-02,
            1.1007547570747180e-02, 1.1026019381147249e-02, 1.1042695595265807e-02, 1.1057860975532409e-02,
            1.1071751857045479e-02, 1.1084565666505860e-02, 1.1096468395525276e-02, 1.1107600514184439e-02,
            1.1118081666841355e-02, 1.1128014432939991e-02, 1.1137487348001939e-02, 1.1146577371101231e-02,
            1.1155351893346113e-02, 1.1163870355731593e-02, 1.1172185618068415e-02, 1.1180345086850615e-02,
            1.1188391761261667e-02, 1.1196364889593627e-02, 1.1204300700566747e-02, 1.1212232963606782e-02,
            1.1220193879270582e-02, 1.1228214214753063e-02, 1.1236323377118574e-02, 1.1244550178188855e-02,
            1.1252923441962841e-02, 1.1261473748637778e-02, 1.1270230850846127e-02, 1.1279221450164715e-02,
            1.1288479658221117e-02, 1.1298037896021292e-02, 1.1307944646122644e-02, 1.1318225812893915e-02,
            1.1328910209404287e-02, 1.1340046911577153e-02, 1.1351721380555410e-02, 1.1364027498287044e-02,
            1.1376911984350185e-02, 1.1390449917960382e-02, 1.1404631513115529e-02, 1.1419998637800782e-02,
            1.1436706611044661e-02, 1.1454320558660740e-02, 1.1472107181507585e-02, 1.1491546656562140e-02,
            1.1511214906719663e-02, 1.1544621281952909e-02, 1.1568033041001595e-02, 1.1596005189903223e-02,
            1.1611886374448196e-02, 1.1642999393803366e-02, 1.1703605655774101e-02, 1.1944617349004275e-02,
            1.1998375088894278e-02, 1.2329022152040849e-02, 1.3931620772305212e-02, 1.7679054162172823e-02,
            2.5992451059429083e-02, 3.9945131495550699e-02
        ], dtype=dtype, device=device)

    elif 0.5 <= alpha <= 0.9:
        gx = torch.tensor([
            3.7609390003375964e-08, 1.3685854979526549e-06, 1.4889428111233842e-05, 8.4909234018965159e-05,
            3.1873821783035098e-04, 8.9204133360438380e-04, 2.0127465688419980e-03, 3.8618683453436136e-03,
            6.5470889986536940e-03, 1.0090726664739097e-02, 1.4447552604036681e-02, 1.9533895674843968e-02,
            2.5252790259180181e-02, 3.1509724331768026e-02, 3.8220152765976879e-02, 4.5311835199784489e-02,
            5.2724547956529318e-02, 6.0408722152233675e-02, 6.8323799499387480e-02, 7.6436655143339735e-02,
            8.4720214965027643e-02, 9.3152294763300211e-02, 1.0171464825117857e-01, 1.1039219699792262e-01,
            1.1917241285088062e-01, 1.2804482522689517e-01, 1.3700062911969721e-01, 1.4603237349945128e-01,
            1.5513371343540050e-01, 1.6429921251385121e-01, 1.7352418487154128e-01, 1.8280456842410028e-01,
            1.9213682268853544e-01, 2.0151784603996090e-01, 2.1094490837374680e-01, 2.2041559602735405e-01,
            2.2992776650187988e-01, 2.3947951105149096e-01, 2.4906912362070266e-01, 2.5869507493505961e-01,
            2.6835599079492339e-01, 2.7805063380973916e-01, 2.8777788796725268e-01, 2.9753674557007820e-01,
            3.0732629613371543e-01, 3.1714571693036520e-01, 3.2699426489914540e-01, 3.3687126977933890e-01,
            3.4677612827216525e-01, 3.5670829904465617e-01, 3.6666729838885320e-01, 3.7665269672921287e-01,
            3.8666411580352911e-01, 3.9670122617260967e-01, 4.0676374501492912e-01, 4.1685143449747197e-01,
            4.2696410186536610e-01, 4.3710159821272931e-01, 4.4726381749484123e-01, 4.5745069603649885e-01,
            4.6766221598520491e-01, 4.7789840652298787e-01, 4.8815934536220495e-01, 4.9844515052405652e-01,
            5.0875598897275465e-01, 5.1909210453586785e-01, 5.2945380107118389e-01, 5.3984142933037105e-01,
            5.5025535726907182e-01, 5.6069612221610865e-01, 5.7116437727195657e-01, 5.8166086578398579e-01,
            5.9218626968330357e-01, 6.0274130153474181e-01, 6.1332726467412169e-01, 6.2394583575131990e-01,
            6.3459902800886914e-01, 6.4528672842975432e-01, 6.5601051151983347e-01, 6.6677339218565990e-01,
            6.7758442124720375e-01, 6.8844699690792643e-01, 6.9935285580563211e-01, 7.1030369804167903e-01,
            7.2128236992493600e-01, 7.3240671216090558e-01, 7.4367772949733946e-01, 7.5519374839089404e-01,
            7.6736343736510493e-01, 7.7970740763740065e-01, 7.9219118119668186e-01, 8.0570157617752125e-01,
            8.2101055899513764e-01, 8.4330359349538586e-01
        ], dtype=dtype, device=device)

        gw = torch.tensor([
            1.7500611765012179e-07, 3.8882557435783513e-06, 2.9954641912492096e-05, 1.2782804549555320e-04,
            3.6976304598021047e-04, 8.1258486789405663e-04, 1.4598247640076655e-03, 2.2565489131548054e-03,
            3.1172626255617855e-03, 3.9619509977401879e-03, 4.7373256373690322e-03, 5.4189419764267398e-03,
            6.0029781163538484e-03, 6.4967663749125951e-03, 6.9120587945015596e-03, 7.2612932414371596e-03,
            7.5558989407095746e-03, 7.8057162802363175e-03, 8.0189406936974315e-03, 8.2022775474879586e-03,
            8.3611605279137139e-03, 8.4999707834225811e-03, 8.6222333472394299e-03, 8.7307842750017125e-03,
            8.8279088186299960e-03, 8.9154535861816365e-03, 8.9949164606365779e-03, 9.0675180782155505e-03,
            9.1342583376412181e-03, 9.1959609393384989e-03, 9.2533084527800825e-03, 9.3068699414011486e-03,
            9.3571227648605662e-03, 9.4044698327488199e-03, 9.4492533078482065e-03, 9.4917655360856907e-03,
            9.5322578036120403e-03, 9.5709473825848993e-03, 9.6080232366267462e-03, 9.6436506619579036e-03,
            9.6779750821076437e-03, 9.7111251615898172e-03, 9.7432153882503012e-03, 9.7743482267823173e-03,
            9.8046159138632880e-03, 9.8341019486747616e-03, 9.8628823752800656e-03, 9.8910269212001307e-03,
            9.9185999146112931e-03, 9.9456610759173381e-03, 9.9722660729993397e-03, 9.9984675891846859e-03,
            1.0024315689563553e-02, 1.0049857967087024e-02, 1.0075140023966186e-02, 1.0100206959935651e-02,
            1.0125102938360671e-02, 1.0149871950774473e-02, 1.0174556113780179e-02, 1.0199197618018014e-02,
            1.0223846640284091e-02, 1.0248548038151891e-02, 1.0273348419043692e-02, 1.0298288470141991e-02,
            1.0323432854489152e-02, 1.0348849701458745e-02, 1.0374601974019832e-02, 1.0400712381956034e-02,
            1.0427243781987490e-02, 1.0454389862197593e-02, 1.0482244702565721e-02, 1.0510876149932735e-02,
            1.0540013946875334e-02, 1.0570289997603391e-02, 1.0601804398556677e-02, 1.0635579476156305e-02,
            1.0670744865279679e-02, 1.0705103070143742e-02, 1.0742678704510395e-02, 1.0784424144377271e-02,
            1.0838880134468544e-02, 1.0885073551248455e-02, 1.0926729010027615e-02, 1.0963337430818136e-02,
            1.1008609571599130e-02, 1.1284106092130687e-02, 1.1237297680636530e-02, 1.1867898115707079e-02,
            1.2386002478015653e-02, 1.2225388580440253e-02, 1.2967800176569934e-02, 1.3961994785569806e-02,
            1.6951270842679722e-02, 3.1804594306850377e-02
        ], dtype=dtype, device=device)
    else:
        raise ValueError("alpha in (0.9, 1.1): método no definido.")
        
  # Escalamiento de cuadratura
    rank_scaling = (-torch.log(torch.tensor(eps, dtype=dtype, device=device))) ** (1.0 / alpha)
    gx_scaled = rank_scaling * gx
    gw_scaled = rank_scaling / torch.pi * gw

    gx_alpha = gx_scaled ** alpha
    exp_term = torch.exp(-gx_alpha)

    z = -beta * torch.tan(torch.pi * alpha / 2)

    # Expansión para cálculo: broadcasting sobre nodos y valores x
    x_col = x.unsqueeze(-1)  # shape: [len(x), 1]
    gx_row = gx_scaled.unsqueeze(0)  # shape: [1, len(gx)]
    gx_alpha_row = gx_alpha.unsqueeze(0)
    exp_row = exp_term.unsqueeze(0)
    gw_row = gw_scaled.unsqueeze(0)

    h = (x_col - z) * gx_row + z * gx_alpha_row
    cos_h = torch.cos(h)

    integrand = gw_row * cos_h * exp_row
    result = integrand.sum(dim=1)  # sum over j

    return result

In [6]:
def stable_sym_pdf_fourier_integral(x, alpha):
    """
    Estima la densidad de una distribución estable simétrica (β = 0) 
    usando el método de integral de Fourier con cuadratura Gaussiana.
    """
    eps = 1e-16
    device = x.device if torch.is_tensor(x) else "cpu"
    dtype = torch.float64

    x = torch.as_tensor(x, dtype=dtype, device=device)
    alpha = torch.as_tensor(alpha, dtype=dtype, device=device)

    gx = torch.tensor([
        2.7148704107693849e-08, 1.1539065574946093e-06, 1.4068944360213799e-05, 8.8528868370045850e-05,
        3.6522895950416606e-04, 1.1242856520043885e-03, 2.7940849388937931e-03, 5.8998374370067466e-03,
        1.0954815404797378e-02, 1.8308965706909420e-02, 2.7904977612154668e-02, 3.8402581477255261e-02,
        4.7620451072357205e-02, 6.0322361829278297e-02, 7.7212543014189644e-02, 9.6567947626676073e-02,
        1.1770638898039337e-01, 1.4023188371095557e-01, 1.6381653230311635e-01, 1.8807833684292100e-01,
        2.1220569852597571e-01, 2.3337594286746213e-01, 2.5067518179499915e-01, 2.7308170043295299e-01,
        2.9923165644921623e-01, 3.2670651619049274e-01, 3.5488605833898679e-01, 3.8356368161790211e-01,
        4.1264154879954701e-01, 4.4206132991284341e-01, 4.7178318417560017e-01, 5.0177774958174326e-01,
        5.3202251254241339e-01, 5.6249992102768298e-01, 5.9319640320449418e-01, 6.2410173262716340e-01,
        6.5520873661416879e-01, 6.8651280590359287e-01, 7.1801277568472022e-01, 7.4971030516593817e-01,
        7.8161424879996133e-01, 8.1374899659119626e-01, 8.4616111090172197e-01, 8.7910800331170069e-01,
        9.1405149366238070e-01, 9.5623769770932743e-01
    ], dtype=dtype, device=device)

    gw = torch.tensor([
        1.3158109639902134e-07, 3.4262543803194494e-06, 2.9760561009721337e-05, 1.4170149519462160e-04,
        4.5818570532242872e-04, 1.1321935880671628e-03, 2.2968035970710459e-03, 4.0027607768915521e-03,
        6.1688253884964774e-03, 8.5398963899097589e-03, 1.0476270652386517e-02, 9.8155372802538974e-03,
        9.9827686069404245e-03, 1.5219506000563725e-02, 1.8284735354731164e-02, 2.0324582923350147e-02,
        2.1888765891758585e-02, 2.3109125676823113e-02, 2.4000778165476255e-02, 2.4409563071531602e-02,
        2.3429581499251423e-02, 1.8132920380634698e-02, 1.9021640675265461e-02, 2.4967741279101181e-02,
        2.6986577522376015e-02, 2.7877980988375722e-02, 2.8450142752054142e-02, 2.8889685328033834e-02,
        2.9256620565013172e-02, 2.9576419596863371e-02, 2.9862450428147645e-02, 3.0122958378570940e-02,
        3.0363656749165541e-02, 3.0588909434433681e-02, 3.0802353645674175e-02, 3.1007120102372672e-02,
        3.1206109628651280e-02, 3.1401812940439804e-02, 3.1598474495187191e-02, 3.1797517989457728e-02,
        3.2015445283346267e-02, 3.2257598848695605e-02, 3.2604667010320047e-02, 3.3454059051937490e-02,
        3.7403084267749798e-02, 4.8035994401521127e-02
    ], dtype=dtype, device=device)

  # Escalamiento de cuadratura
    rank_scaling = (-torch.log(torch.tensor(eps, dtype=dtype, device=device))) ** (1.0 / alpha)
    gx_scaled = rank_scaling * gx
    gw_scaled = rank_scaling / torch.pi * gw

    gx_alpha = gx_scaled ** alpha
    exp_term = torch.exp(-gx_alpha)

    # Broadcasting: [len(x), 1] vs [1, len(gx)]
    x_col = x.unsqueeze(-1)
    gx_row = gx_scaled.unsqueeze(0)
    gw_row = gw_scaled.unsqueeze(0)
    exp_row = exp_term.unsqueeze(0)

    integrand = gw_row * torch.cos(x_col * gx_row) * exp_row
    result = integrand.sum(dim=1)

    return result

In [7]:
# Versión PyTorch de ament_oneill_pdf, en float64 y replicando el flujo de R
def ament_oneill_pdf(x, alpha, beta, device=None, verbose=True):
    eps = 1e-16

    x = torch.as_tensor(x, dtype=torch.float64)
    alpha = torch.as_tensor(alpha, dtype=torch.float64)
    beta = torch.as_tensor(beta, dtype=torch.float64)

    if device is None:
        device = x.device

    pi = torch.tensor(torch.pi, dtype=torch.float64, device=device)
    x = x.to(device)
    alpha = alpha.to(device)
    beta = beta.to(device)

    z = -beta * torch.tan(pi * alpha / 2)

    if beta.item() == 0:
        n = 42
        n_tensor = torch.tensor(n, dtype=torch.float64, device=device)

        min_inf_x = ((alpha / (pi * eps)) * torch.exp(torch.lgamma(alpha * n_tensor) - torch.lgamma(n_tensor))) ** (1 / (alpha * n_tensor - 1))
        x_abs = x.abs()

        inf_cond = x_abs > min_inf_x
        fourier_cond = ~inf_cond

        pdf = torch.zeros_like(x, dtype=torch.float64)

        if inf_cond.any():
            pdf[inf_cond] = stable_pdf_series_infinity(x_abs[inf_cond], alpha, beta, n, device=device)

        if fourier_cond.any():
            pdf[fourier_cond] = stable_sym_pdf_fourier_integral(x_abs[fourier_cond], alpha)

        return pdf

    else:
        alpha_val = alpha.item()
        if 0.5 <= alpha_val <= 0.9:
            n = 90
        elif 1.1 <= alpha_val <= 2.0:
            n = 80
        else:
            raise ValueError("Alpha fuera de rango permitido para Ament-O'Neill")

        pdf = torch.full_like(x, torch.nan, dtype=torch.float64)
        xlz_cond = x < z

        if xlz_cond.any():
            pdf[xlz_cond] = ament_oneill_pdf(-x[xlz_cond], alpha, -beta, device=device, verbose=verbose)

        if (~xlz_cond).any():
            x_sub = x[~xlz_cond]
            n_tensor = torch.tensor(n, dtype=torch.float64, device=device)
            exp_term = torch.exp(torch.lgamma(alpha * n_tensor) - torch.lgamma(n_tensor))

            base = (1 + z**2) ** (n_tensor / 2) * (alpha / (pi * eps)) * exp_term
            denom = alpha * n_tensor - 1
            min_inf_x = base ** (1 / denom) + z

            inf_cond = x_sub > min_inf_x
            fourier_cond = (x_sub > z) & (~inf_cond)

            indices = torch.where(~xlz_cond)[0]

            if inf_cond.any():
                res_inf = stable_pdf_series_infinity(x_sub[inf_cond], alpha, beta, n, device=device)
                pdf[indices[inf_cond]] = res_inf

            if fourier_cond.any():
                res_fourier = stable_pdf_fourier_integral(x_sub[fourier_cond], alpha, beta)
                pdf[indices[fourier_cond]] = res_fourier

        return pdf


In [8]:
def stablepdf(x, alpha, beta, verbose=True):
    # Validación de parámetros
    alpha_val = alpha.item() if torch.is_tensor(alpha) else alpha
    beta_val = beta.item() if torch.is_tensor(beta) else beta

    if (not 0.1 <= alpha_val <= 2) or (abs(beta_val) > 1):
        raise ValueError("Error: Parámetros inválidos")

    # Caso 1: Distribución Normal (α = 2, β = 0)
    elif alpha_val == 2 and beta_val == 0:
        if verbose: print("Usando normal")
        vals = norm.pdf(x.detach().cpu().numpy(), loc=0, scale=1)
        return torch.from_numpy(vals).to(x.device).to(dtype=x.dtype)

    # Caso 2: Distribución Cauchy (α = 1)
    elif alpha_val == 1:
        if verbose: print("Usando Cauchy")
        vals = cauchy.pdf(x.detach().cpu().numpy(), loc=0, scale=1)
        return torch.from_numpy(vals).to(x.device).to(dtype=x.dtype)

    # Caso 3: α ≤ 0.5 → usar Nolan sin gradiente
    elif alpha_val <= 0.5:
        if verbose: print(f"Usando Nolan (α ≤ 0.5, sin gradiente) — α={alpha_val:.4f}, β={beta_val:.4f}")
        vals = levy_stable.pdf(x.detach().cpu().numpy(), alpha_val, beta_val)
        val = torch.from_numpy(vals).to(x.device).to(dtype=x.dtype)
        dummy = (alpha * 0 + beta * 0 + x[0])  # dummy conectado al grafo
        return val + 0.0 * dummy

    # Caso 4: Ament y O'Neill con β = 0 (simétrica)
    elif beta_val == 0:
        if verbose: print("Ament-O’Neill simétrica")
        return ament_oneill_pdf(x, alpha, beta)

    # Caso 5: α cercano a 1 con β ≠ 0 → Nolan sin gradiente
    elif 0.9 < alpha_val < 1.1 and beta_val != 0:
        if verbose: print("Usando Nolan (zona crítica, sin gradiente)")
        vals = levy_stable.pdf(x.detach().cpu().numpy(), alpha_val, beta_val)
        val = torch.from_numpy(vals).to(x.device).to(dtype=x.dtype)
        dummy = (alpha * 0 + beta * 0 + x[0])  # dummy conectado
        return val + 0.0 * dummy

    # Caso 6: Ament y O'Neill general (asimétrica)
    else:
        if verbose: print("Ament-O’Neill asimétrica")
        return ament_oneill_pdf(x, alpha, beta)


In [9]:
class AICLossOnly(nn.Module):
    def __init__(self, stablepdf, k=2, penalty=1e6, verbose=True):
        super().__init__()
        self.stablepdf = stablepdf
        self.k = k
        self.penalty = penalty
        self.verbose = verbose

    def forward(self, residuos_batch, alpha_beta_pred, alpha_beta_true, return_vector=False):
        alpha = alpha_beta_pred[:, 0]
        beta = alpha_beta_pred[:, 1]

        loss_vals = alpha * 0.0  # mismo shape, dtype y grafo

        try:
            pdf_vals_list = []

            for i in range(residuos_batch.size(0)):
                x = residuos_batch[i]
                a = alpha[i]
                b = beta[i]
                pdf_i = self.stablepdf(x, a, b, verbose=self.verbose)
                pdf_vals_list.append(pdf_i)

            pdf_vals = torch.stack(pdf_vals_list)  # (batch_size, len_serie)

            # Penalización si hay problemas
            invalid_mask = ~torch.isfinite(pdf_vals) | (pdf_vals <= 0)
            if invalid_mask.any():
                if self.verbose:
                    print(f"{invalid_mask.sum().item()} muestras AIC inválidas penalizadas")
                loss_vals = torch.full_like(loss_vals, self.penalty)
            else:
                pdf_vals = torch.clamp(pdf_vals, min=1e-8)
                logliks = torch.sum(torch.log(pdf_vals), dim=1)
                aics = 2 * self.k - 2 * logliks
                loss_vals = aics.to(loss_vals.dtype)

                if self.verbose:
                    print(f"[AICLossOnly] AIC aplicado en {aics.size(0)} muestras")

        except Exception as e:
            if self.verbose:
                print(f"Error global al aplicar AIC: {e}")
            loss_vals = torch.full_like(loss_vals, self.penalty)

        loss_mean = loss_vals.mean()
        return (loss_mean, loss_vals) if return_vector else loss_mean


In [10]:
def run_pretrain_loop(model, dataloader, optimizer, criterion,
                      epochs=10, device=None, save_name="modelo_pretrain",
                      patience=5, min_delta=0.001, checkpoint_path=None):

    import os, torch
    from tqdm import tqdm

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device
    model.to(device).double()

    losses, mae_alphas, mae_betas = [], [], []

    def calcular_mae(real, pred):
        return torch.mean(torch.abs(real - pred)).item()

    # early stopping
    best_loss = float("inf")
    epochs_no_improve = 0
    start_epoch = 0

    # if checkpoint dale
    if checkpoint_path and os.path.exists(checkpoint_path):
        ckpt = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(ckpt["model_state_dict"])
        optimizer.load_state_dict(ckpt["optimizer_state_dict"])
        best_loss = ckpt.get("best_loss", float("inf"))
        epochs_no_improve = ckpt.get("epochs_no_improve", 0)
        start_epoch = ckpt.get("epoch", -1) + 1
        print(f"Reanudando desde epoch {start_epoch} (mejor loss {best_loss:.6f})")

    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss, total_samples = 0.0, 0
        all_alpha_real, all_beta_real = [], []
        all_alpha_pred, all_beta_pred = [], []

        skipped_nonfinite = 0
        processed_batches = 0

        for residuos_batch, targets, _ in tqdm(dataloader, desc=f"[Pretrain] Epoch {epoch+1}/{epochs}"):
            residuos_batch = residuos_batch.to(device).double()

            if isinstance(targets, list):
                targets = torch.stack([torch.as_tensor(t) if not isinstance(t, torch.Tensor) else t for t in targets])
            targets = targets.to(device).double()

            optimizer.zero_grad(set_to_none=True)
            pred_params = model(residuos_batch)
            loss, _ = criterion(residuos_batch, pred_params, targets, return_vector=True)

            if not torch.isfinite(loss):
                skipped_nonfinite += 1
                continue

            loss.backward()
            #parche
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            bsz = residuos_batch.size(0)
            total_loss += loss.item() * bsz
            total_samples += bsz
            processed_batches += 1

            with torch.no_grad():
                all_alpha_real.append(targets[:, 0].detach().cpu())
                all_beta_real.append(targets[:, 1].detach().cpu())
                all_alpha_pred.append(pred_params[:, 0].detach().cpu())
                all_beta_pred.append(pred_params[:, 1].detach().cpu())

        if processed_batches == 0 or total_samples == 0:
            # epoch fail
            print(f"Epoch {epoch+1}: todas las pérdidas fueron no finitas "
                  f"(skipped={skipped_nonfinite}). Reducir lr / revisar datos.")
            epoch_loss = float("inf")
            mae_alpha = float("nan")
            mae_beta  = float("nan")
        else:
            all_alpha_real = torch.cat(all_alpha_real)
            all_beta_real  = torch.cat(all_beta_real)
            all_alpha_pred = torch.cat(all_alpha_pred)
            all_beta_pred  = torch.cat(all_beta_pred)

            mae_alpha = calcular_mae(all_alpha_real, all_alpha_pred)
            mae_beta  = calcular_mae(all_beta_real,  all_beta_pred)
            epoch_loss = total_loss / max(total_samples, 1)

        losses.append(epoch_loss)
        mae_alphas.append(mae_alpha)
        mae_betas.append(mae_beta)

        # apply early stopping logic
        if not torch.isfinite(torch.tensor(epoch_loss)):
            delta = float("nan")
        elif best_loss == float("inf"):
            # epoch 1
            best_loss = epoch_loss
            epochs_no_improve = 0
            delta = float("inf")
            print(f"💾 Primer baseline guardado (loss: {best_loss:.6f})")
        else:
            denom = max(abs(best_loss), 1e-12)
            delta = (best_loss - epoch_loss) / denom
            if delta > min_delta:
                best_loss = epoch_loss
                epochs_no_improve = 0
                print(f"💾 Nuevo mejor modelo (loss: {best_loss:.6f}) — mejora: {delta:.2%}")
            else:
                epochs_no_improve += 1
                print(f"⚠️ Sin mejora sustancial ({delta:.2%}) — {epochs_no_improve}/{patience}")

        # checkpoint save
        if checkpoint_path:
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_loss": best_loss,
                "epochs_no_improve": epochs_no_improve
            }, checkpoint_path)

        # stop condition
        if epochs_no_improve >= patience:
            print(f"Early stopping activado en epoch {epoch+1}")
            break

    # parche
    if len(losses) == 0:
        return {"loss_mean": float("inf"), "mae_alpha": float("nan"),
                "mae_beta": float("nan"), "best_loss": float("inf")}

    return {
        "loss_mean": float(sum(x for x in losses if x == x)) / max(len([x for x in losses if x == x]), 1),
        "mae_alpha": float(sum(x for x in mae_alphas if x == x)) / max(len([x for x in mae_alphas if x == x]), 1),
        "mae_beta":  float(sum(x for x in mae_betas  if x == x)) / max(len([x for x in mae_betas  if x == x]), 1),
        "best_loss": float(best_loss)
    }


In [11]:
import torch
import torch.nn as nn
#RNN
def get_model(input_dim, hidden_size=64, dropout_rate=0.2,
              num_layers=1, bidirectional=False):
    class RNN(nn.Module):
        def __init__(self, input_dim, hidden_size, dropout_rate, num_layers, bidirectional):
            super().__init__()
            self.bidirectional = bidirectional
            self.rnn = nn.LSTM(
                input_size=1,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout_rate if num_layers > 1 else 0.0,
                bidirectional=bidirectional
            )
            fc_input_size = hidden_size * (2 if bidirectional else 1)
            self.fc = nn.Linear(fc_input_size, 2)

        def forward(self, x):
            # [batch, seq_len] → [batch, seq_len, 1]
            x = x.unsqueeze(-1)
            out, _ = self.rnn(x)
            last_output = out[:, -1, :]  # última salida de la secuencia
            out = self.fc(last_output)

            # ALPHA en [1.1, 2], BETA en (-1, 1)
            alpha_out = torch.clamp(torch.sigmoid(out[:, 0]), min=1e-6, max=1.0 - 1e-6)
            beta_out = torch.clamp(torch.tanh(out[:, 1]), min=-0.999, max=0.999)

            eps = 1e-4
            alpha = (1.1 + eps) + alpha_out * ((2.0 - eps) - (1.1 + eps))
            beta = beta_out
            return torch.stack([alpha, beta], dim=1)

    return RNN(input_dim, hidden_size, dropout_rate, num_layers, bidirectional)

In [12]:
#pretrain
def run_pretrain_gridsearch(architecture, param_grid, input_dim,
                            dataloader, criterion_class, optimizer_class,
                            lr_key="lr", epochs=15, device="cpu", top_n=50,
                            checkpoint_dir="checkpoints2"):

    import itertools
    import pandas as pd
    import os

    os.makedirs(checkpoint_dir, exist_ok=True)

    keys = list(param_grid.keys())
    combos = list(itertools.product(*[param_grid[k] for k in keys]))

    resultados = []

    for i, combo in enumerate(combos):
        print(f"\n Pretrain combo {i+1}/{len(combos)} — {architecture}")
        params = dict(zip(keys, combo))
        model_params = {k: v for k, v in params.items()
                        if k in ["hidden_size", "dropout_rate", "num_layers", "bidirectional"]}
        lr = params.get(lr_key, 1e-4)
        weight_decay = params.get("weight_decay", 0.0)

        combo_id = "_".join([f"{k}{v}" for k, v in params.items()])
        save_name = f"{architecture}_pretrain_{combo_id}"

        # checkpoint combinación
        checkpoint_path = os.path.join(checkpoint_dir, f"{save_name}.pth")

        model = get_model(input_dim, **model_params).to(device).double()
        optimizer = optimizer_class(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = criterion_class(stablepdf, verbose=False)

        resultado = run_pretrain_loop(
            model=model,
            dataloader=dataloader,
            optimizer=optimizer,
            criterion=criterion,
            epochs=epochs,
            device=device,
            save_name=save_name,
            patience=5,
            min_delta=0.001,
            checkpoint_path=checkpoint_path   # añadimos checkpoint persistente
        )

        resultado.update({
            "architecture": architecture,
            "aic_mean": resultado["loss_mean"],
            **params
        })
        resultados.append(resultado)

    # Guardar resultado
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(f"gridsearch_pretrain_resultados_{architecture}.csv", index=False)

    # Extraer top-N por best_loss
    df_top = df_resultados.sort_values("best_loss").head(top_n)
    df_top.to_csv("top50_pretrain.csv", index=False)
    print(f"\n Pretrain finalizado. Top {top_n} guardado en top50_pretrain.csv")

    return df_top


In [13]:
#GRID
param_grid_rnn = {
    "hidden_size": [32, 64, 128, 256],
    "num_layers": [1, 2, 3],
    "dropout_rate": [0.0, 0.1, 0.2, 0.3],
    "bidirectional": [False, True],
    "lr": [1e-2, 1e-3, 1e-4, 5e-5],
    "weight_decay": [0.0, 1e-5, 1e-4]
}

In [14]:
#RUNNER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def main_pretrain_runner():
    input_dim = next(iter(loader_pretrain))[0].shape[1]
    print("\nIniciando GridSearch Pretrain para RNN")
    df_top_rnn = run_pretrain_gridsearch(
        architecture="RNN",
        param_grid=param_grid_rnn,
        input_dim=input_dim,
        dataloader=loader_pretrain,
        criterion_class=AICLossOnly,
        optimizer_class=torch.optim.Adam,
        device=device,
        epochs=10,
        top_n=50
    )

    # === Guardar resumen global de los mejores modelos ===
    df_top_rnn.assign(model="RNN").to_csv("top50_pretrain_TODOS.csv", index=False)
    print("\n Pretrain finalizado. Top 50 guardado en top50_pretrain_TODOS.csv")
    return df_top_rnn

In [15]:
main_pretrain_runner()

KeyboardInterrupt: 